Verify LOOCV derivative equations via finite differences

In [1]:
import numpy as np
import scipy.misc
import pandas as pd
np.random.seed(0)
n, k = 25, 3
sigma = 0.1
X = np.random.random_sample((n,k))
beta = np.random.random_sample(k)
y = np.dot(X, beta) + np.random.normal(scale=sigma, size=n)
Gamma = np.diag(np.random.random_sample(k))

In [2]:
def compute_loocv(X, y, Gamma):
    A = np.dot(X.T, X) + np.dot(Gamma.T, Gamma)
    A_inv = np.linalg.inv(A)
    b_hat = np.dot(A_inv, np.dot(X.T, y))
    y_hat = np.dot(X, b_hat)
    h = np.array([np.dot(x_i, np.dot(A_inv, x_i)) for x_i in X])
    return np.sum(((y - y_hat) / (1 - h))**2)

In [3]:
def compute_loocv_derivative(X, y, Gamma, s):
    A = np.dot(X.T, X) + np.dot(Gamma.T, Gamma)
    A_inv = np.linalg.inv(A)
    b_hat = np.dot(A_inv, np.dot(X.T, y))
    y_hat = np.dot(X, b_hat)
    h = np.array([np.dot(x_i, np.dot(A_inv, x_i)) for x_i in X])
    alpha_s = Gamma[s, s]
    X_A_inv = np.dot(X, A_inv)
    A_inv_X_t_y = np.dot(A_inv, np.dot(X.T, y))
    result = 0
    for i in range(len(y)):
        e_hat_i = (y[i] - y_hat[i]) / (1 - h[i])
        d_y_hat_i = -2 * alpha_s * X_A_inv[i, s] * A_inv_X_t_y[s]
        d_h_i = -2 * alpha_s * X_A_inv[i, s]**2
        d_e_hat_i = (y[i] - y_hat[i]) / (1 - h[i])**2*d_h_i - d_y_hat_i / (1 - h[i])
        result += 2*e_hat_i * d_e_hat_i
    return result

In [4]:
def compute_loocv_approx_derivative(X, y, Gamma, s):
    def f(x):
        Gamma_prime = np.array(Gamma)
        Gamma_prime[s, s] = x
        return compute_loocv(X, y, Gamma_prime)
    return scipy.misc.derivative(f, Gamma[s,s], dx=np.sqrt(np.finfo(float).eps))

In [5]:
derivatives_approx = [compute_loocv_approx_derivative(X, y, Gamma, s) for s in range(k)]

In [6]:
derivatives = [compute_loocv_derivative(X, y, Gamma, s) for s in range(k)]

In [7]:
pd.DataFrame({'approx' : derivatives_approx, 'computed' : derivatives})

,approx,computed
0,-0.005690,-0.005690
1,0.019269,0.019269
2,-0.005988,-0.005988
